### Цели и задачи проекта

- Помочь коллегам из «Секретов Темнолесья» изучить развитие игровой индустрии. 
- Изучить датасет  о продажах играх.
- Проверить корректность и провести предобработку, получив необходимый срез данных.

### Описание данных

- Name — название игры.
- Platform — название платформы.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах проданных копий).
- EU sales — продажи в Европе (в миллионах проданных копий).
- JP sales — продажи в Японии (в миллионах проданных копий).
- Other sales — продажи в других странах (в миллионах проданных копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

- Ознакомиться и изучить данные.
- Проверить данные на наличие ошибок и провести предобработку.
- Отфильтровать данные.
- Провести категоризацию данных.
- Сделать выводы по проделанной работе.

---

## 1. Загрузка данных и знакомство с ними

Загрузим необходимые данные и сам датасет.


In [137]:
import pandas as pd

In [ ]:
df = pd.read_csv('')

Выведем первые строки датасета

In [139]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Выведем всю информацию о датасете

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Запомним количество строк в датасете

In [141]:
first = df.shape[0]
first

16956

Изучим типы данных и их корректность:
- **Числовые значения с плавающей запятой (float64).** Четыре столбеца, `Year of Release`, содержит год выпуска игры, стоит преобразовать его либо в формат даты, либо в integer, так как у нас информация содержит только год, а не полную дату. `NA sales` - продажи в Северной америке, `Other sales` - продажи в других странах, `Critic Score` - оценка критиков, для этих столбцов представлен верный тип данных так как продажи и оценка погут иметь не целочисленные значения.
- **Строковые данные (object).** Семь столбцов имеют тип данных `object`:
    - `Name`, `Platform` содержат строковую информацию (название игры и платформы ), что логично для текстовых данных. Здесь тип данных `object` подходит.
    - `EU sales`,`JP sales`, `User Score` хранят информацию о продажах и оценках пользователей. Для таких данных рекомендуется использовать тип `float64`, который облегчит выполнение числовых вычислений.
    - `Genre`, `Rating` также хранят текстовые данные, но их можно рассматривать как категориальные признаки. В этом случае можно использовать тип `category`, чтобы улучшить производительность и оптимизировать память, если набор значений ограничен и заведомо известен.

Всего в датасете 16954 строки и 11 колонок, всего 2 типа данных: object, float64. Можно заметить, что данные содержат пропуски.  Также есть необходимость привести все столбцы к стилю snake case.

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма


Посмотрим на название всех столбцоф датафрейма

In [142]:
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

Заметно, что необходимо привести названия столбцов к snake case.

In [143]:
df = df.rename(columns={'Year of Release': 'year_of_release', 'NA sales': 'na_sales', 'EU sales': 'eu_sales', 'JP sales': 'jp_sales', 'Other sales': 'other_sales', 'Critic Score': 'critic_score', 'User Score': 'user_score'})
df.columns = df.columns.str.lower()

Посмотрим на получившийся результат

In [144]:
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных



Проверим аномальные значения в числовых столбцах. Найдем столбцы, где встречается какое-то строковое значение типа `unknown`. Выведем название столбцов и аномальные значения

In [145]:
def get_non_numbers(data, column):
    unique_values = data[column].dropna().unique()
    
    result = []
    for value in unique_values:
        try:
            float(value)
        except:
            result.append(value)
    
    return result

In [146]:
for column in ['critic_score', 'user_score', 'eu_sales', 'jp_sales', 'year_of_release', 'na_sales', 'other_sales']:
    print(f'{column}: {get_non_numbers(df, column)}')

critic_score: []
user_score: ['tbd']
eu_sales: ['unknown']
jp_sales: ['unknown']
year_of_release: []
na_sales: []
other_sales: []


Можно заметить, что в столбце 'user_score' хранится неккоректное значение 'tbd', а в столбцах 'eu_sales' и 'jp_sales' есть значения 'unknown'. Заменим их на пропуски и изменим тип столбца на float64

In [147]:
df[['eu_sales', 'jp_sales','user_score']] = df[['eu_sales', 'jp_sales','user_score']].apply(pd.to_numeric, errors='coerce')

Преобразуем столбец year_of_release к типу int

In [148]:
df['year_of_release'] = pd.to_numeric(df['year_of_release'], downcast = 'integer', errors = 'coerce')

Проверим, что все наши изменения применились.

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


Все столбики приведены к корректным типам

### 2.3. Наличие пропусков в данных

Посчитаем количество пропусков в каждом столбце в абсолютных. 


In [150]:
df.isnull().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

Посчитаем количество пропусков в каждом столбце в относительных значениях.

In [151]:
df.isnull().sum()/len(df) * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Удалим пропуски в столбцах name, genre, year_of_release так как там встречается незначительный процент пропусков. И мы не можем корректно заменить пропуски


In [152]:
df = df.dropna(subset = ['name', 'genre', 'year_of_release'])

Заменим пропуски в региональных продажах средним сгруппированном по году и платформе

In [153]:
def sales(row):
    if pd.isna(row['eu_sales']):
        group = df[(df['year_of_release'] == row['year_of_release']) & 
                   (df['platform'] == row['platform'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']
def sales(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['year_of_release'] == row['year_of_release']) & 
                   (df['platform'] == row['platform'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']

df['eu_sales'] = df.apply(sales, axis=1)
df['jp_sales'] = df.apply(sales, axis=1)


Много пропусков встречаются в таких столбцах, как 'critic_score', 'user_score', 'rating', пропусков практически 50%. Можно  выдвинуть гипотезу: для слишком старых игр или для игр из других регионов, не переведенных на английский язык, не существует достаточного количества оценок критиков и пользователей по естественным причинам. 

Заменим пропущенные значения в столбике 'rating' на Неизвестно

In [154]:
df['rating'] = df['rating'].fillna('Неизвестно')


Заменим пропущенные значения в столбиках 'critic_score', 'user_score' на нереалистичное значение -1

In [155]:
df[['critic_score', 'user_score']] = df[['critic_score', 'user_score']].fillna(-1)

In [156]:
df.isnull().sum()

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

### 2.4. Явные и неявные дубликаты в данных



Выведем все уникальные значения столбцов, в которых могли бы встречаться дубликаты

In [157]:
print(f'genre: {df['genre'].unique()}\n')
print(f'platform: {df['platform'].unique()}\n')
print(f'rating: {df['rating'].unique()}\n')

genre: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']

platform: ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']

rating: ['E' 'Неизвестно' 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']



В столбцах могт быть дубликаты из-за разного регистра букв. Попробуем привести все значения колонок к нижнему регистру.

In [158]:

df['genre'] = df['genre'].str.lower()
df['platform'] = df['platform'].str.lower()
df['rating'] = df['rating'].str.lower()

In [159]:
print(f'genre: {df['genre'].unique()}\n')
print(f'platform: {df['platform'].unique()}\n')
print(f'rating: {df['rating'].unique()}\n')

genre: ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']

platform: ['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']

rating: ['e' 'неизвестно' 'm' 't' 'e10+' 'k-a' 'ao' 'ec' 'rp']



В столбце genre записей стало меньше. Однако в столбце rating есть похожие значения, которые использовались в разные временные промежутки, заменим их

In [160]:
df['rating'] = df['rating'].replace({'k-a': 'e', 'ec': 'e'})
print(f'rating: {df['rating'].unique()}')

rating: ['e' 'неизвестно' 'm' 't' 'e10+' 'ao' 'rp']


Вариантов значений в колонке рейтинг стало меньше

Посмотрим, сколько в датасете дубликатов

In [161]:
drop = df.duplicated().sum()
drop

np.int64(235)

Удалим их и посчитаем количество удалённых строк в абсолютном и относительном значениях от начального датасета

In [162]:
df = df.drop_duplicates()

In [163]:
print(first/df.shape[0])
print(first - df.shape[0])

1.0311359766480175
512


После предобработки данных можно заметить, что датасет стал чише, он не содержит дубликатов и пропусков, что удобно для дальнейшего анализа. Было удалено 516 неккоректных или дублирующих записей, что составили чуть больше 1%, что является допустимым значением

---

## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Поэтому нужно отобрать данные по этому показателю. Сохраним новый срез данных в отдельном датафрейме `df_actual`.

In [164]:
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,wii,2006.0,sports,41.36,3.77,3.77,8.45,76.0,8.0,e
2,Mario Kart Wii,wii,2008.0,racing,15.68,3.79,3.79,3.29,82.0,8.3,e
3,Wii Sports Resort,wii,2009.0,sports,15.61,3.28,3.28,2.95,80.0,8.0,e
6,New Super Mario Bros.,ds,2006.0,platform,11.28,6.50,6.50,2.88,89.0,8.5,e
7,Wii Play,wii,2006.0,misc,13.96,2.93,2.93,2.84,58.0,6.6,e
...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,gc,2003.0,shooter,0.01,0.00,0.00,0.00,-1.0,-1.0,t
16949,Woody Woodpecker in Crazy Castle 5,gba,2002.0,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,неизвестно
16950,SCORE International Baja 1000: The Official Game,ps2,2008.0,racing,0.00,0.00,0.00,0.00,-1.0,-1.0,неизвестно
16952,LMA Manager 2007,x360,2006.0,sports,0.00,0.00,0.00,0.00,-1.0,-1.0,неизвестно


В датасете осталось всего 12776 строк для анализа

---

## 4. Категоризация данных
    
Проведем категоризацию данных:
- Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала). Однако все пропуски в пользовательской оценке мы заменили нулями, поэто выделим отдельную категорию для нулей - Неуказанно

In [165]:
def categorize(row):
    if row['user_score'] >= 8 and row['user_score']<=10:
        return "Высокая оценка"
    elif row['user_score'] >= 3 and row['user_score'] < 8:
        return "Средняя оценка"
    elif row['user_score'] >= 0 and row['user_score'] < 3:
        return "Низкая оценка"
    else:
        return "Неуказано"

df_actual['category_user'] = df_actual.apply(categorize, axis=1)
df_actual.head(10)

C:\Users\miss-\AppData\Local\Temp\ipykernel_24404\3514276064.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['category_user'] = df_actual.apply(categorize, axis=1)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,category_user
0,Wii Sports,wii,2006.0,sports,41.36,3.77,3.77,8.45,76.0,8.0,e,Высокая оценка
2,Mario Kart Wii,wii,2008.0,racing,15.68,3.79,3.79,3.29,82.0,8.3,e,Высокая оценка
3,Wii Sports Resort,wii,2009.0,sports,15.61,3.28,3.28,2.95,80.0,8.0,e,Высокая оценка
6,New Super Mario Bros.,ds,2006.0,platform,11.28,6.50,6.50,2.88,89.0,8.5,e,Высокая оценка
7,Wii Play,wii,2006.0,misc,13.96,2.93,2.93,2.84,58.0,6.6,e,Средняя оценка
8,New Super Mario Bros. Wii,wii,2009.0,platform,14.44,4.70,4.70,2.24,87.0,8.4,e,Высокая оценка
10,Nintendogs,ds,2005.0,simulation,9.05,1.93,1.93,2.74,-1.0,-1.0,неизвестно,Неуказано
11,Mario Kart DS,ds,2005.0,racing,9.71,4.13,4.13,1.90,91.0,8.6,e,Высокая оценка
13,Wii Fit,wii,2007.0,sports,8.92,3.60,3.60,2.15,80.0,7.7,e,Средняя оценка
14,Kinect Adventures!,x360,2010.0,misc,15.00,0.24,0.24,1.69,61.0,6.3,e,Средняя оценка


- Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала). Однако все пропуски в оценке критиков мы заменили нулями, поэто выделим отдельную категорию для нулей - Неуказанно

In [166]:
def categorize(row):
    if row['critic_score'] >= 80 and row['critic_score']<=100:
        return "Высокая оценка"
    elif row['critic_score'] >= 30 and row['critic_score'] < 80:
        return "Средняя оценка"
    elif row['critic_score'] >= 0 and row['critic_score'] < 30:
        return "Низкая оценка"
    else:
        return "Неуказано"

df_actual['category_critic'] = df_actual.apply(categorize, axis=1)
df_actual.head(10)

C:\Users\miss-\AppData\Local\Temp\ipykernel_24404\3243254990.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['category_critic'] = df_actual.apply(categorize, axis=1)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,category_user,category_critic
0,Wii Sports,wii,2006.0,sports,41.36,3.77,3.77,8.45,76.0,8.0,e,Высокая оценка,Средняя оценка
2,Mario Kart Wii,wii,2008.0,racing,15.68,3.79,3.79,3.29,82.0,8.3,e,Высокая оценка,Высокая оценка
3,Wii Sports Resort,wii,2009.0,sports,15.61,3.28,3.28,2.95,80.0,8.0,e,Высокая оценка,Высокая оценка
6,New Super Mario Bros.,ds,2006.0,platform,11.28,6.50,6.50,2.88,89.0,8.5,e,Высокая оценка,Высокая оценка
7,Wii Play,wii,2006.0,misc,13.96,2.93,2.93,2.84,58.0,6.6,e,Средняя оценка,Средняя оценка
8,New Super Mario Bros. Wii,wii,2009.0,platform,14.44,4.70,4.70,2.24,87.0,8.4,e,Высокая оценка,Высокая оценка
10,Nintendogs,ds,2005.0,simulation,9.05,1.93,1.93,2.74,-1.0,-1.0,неизвестно,Неуказано,Неуказано
11,Mario Kart DS,ds,2005.0,racing,9.71,4.13,4.13,1.90,91.0,8.6,e,Высокая оценка,Высокая оценка
13,Wii Fit,wii,2007.0,sports,8.92,3.60,3.60,2.15,80.0,7.7,e,Средняя оценка,Высокая оценка
14,Kinect Adventures!,x360,2010.0,misc,15.00,0.24,0.24,1.69,61.0,6.3,e,Средняя оценка,Средняя оценка


После категоризации данных проверим результат.

Сгруппируем данные по выделенной категории оценки пользователей и посчитаем количество игр в каждой категории.

In [167]:
grouped_data_user = df_actual.groupby('category_user')['name'].count()
print(grouped_data_user) 

category_user
Высокая оценка    2286
Неуказано         6298
Низкая оценка      116
Средняя оценка    4081
Name: name, dtype: int64


Сгруппируем данные по выделенной категории оценки критиков и посчитаем количество игр в каждой категории.

In [168]:
grouped_data_critic = df_actual.groupby('category_critic')['name'].count()
print(grouped_data_critic) 

category_critic
Высокая оценка    1692
Неуказано         5612
Низкая оценка       55
Средняя оценка    5422
Name: name, dtype: int64


Сгруппируем данные по выделенной категории оценки пользователей и критиков и посчитаем количество игр в каждой категории.

In [169]:
grouped_data = df_actual.groupby(['category_user', 'category_critic'])['name'].count()
print(grouped_data) 

category_user   category_critic
Высокая оценка  Высокая оценка     1017
                Неуказано           101
                Низкая оценка         1
                Средняя оценка     1167
Неуказано       Высокая оценка       33
                Неуказано          5237
                Низкая оценка         7
                Средняя оценка     1021
Низкая оценка   Высокая оценка        1
                Неуказано            21
                Низкая оценка        17
                Средняя оценка       77
Средняя оценка  Высокая оценка      641
                Неуказано           253
                Низкая оценка        30
                Средняя оценка     3157
Name: name, dtype: int64


Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [170]:
grouped_data_platform = df_actual.groupby('platform')['name'].count()
top7_platforms = grouped_data_platform.sort_values(ascending=False).head(7)
print(top7_platforms)


platform
ps2     2127
ds      2120
wii     1275
psp     1180
x360    1121
ps3     1087
gba      811
Name: name, dtype: int64


---

## 5. Итоговый вывод


Были загружены данные `new_games.csv`. Они содержат 11 столбцов и 16954 строк, в которых представлена информация о продажах компьютерных игр. Столбцы были переименованы под стандарт snake case. При первичном знакомстве с данными и их предобработкой получили такие результаты:
- В восьми столбцах (`name`, `year_of_release`, `genre`, `eu_sales`,`jp_sales`, `critic_score`, `user_score`, `rating`) были обнаружены пропущенные значения. Максимальное значение пропущенных данных в столбце `user_score` почти 55%.
- Для оптимизации работы с данными в датафрейме были произведены следующие изменения типов данных:
    - `year_of_release`: тип данных изменён с `float64` на `int64`.
    - `eu_sales`, `jp_sales` и `children`: тип данных изменён с `object` на `float64`.
- Для дополнительной работы с данными были созданы дополнительные столбцы:
    - `category_user`: столбец с отформатироваными данными об оценко пользователей в формате: высокая оценка, средняя оценка, низкая оценка и неуказано;
    - `category_critic`: столбец с отформатироваными данными об оценко критиков в формате: высокая оценка, средняя оценка, низкая оценка и неуказано;
- В данных была проведена работа по удалению дубликатов и пропуско, было удалено 522 строки. Таккже были заменены пропущенные значения, подходящие по смысловой нагрузке.
- Данные были отфильтрованы по году от 2000 до 2013 и по ним был проведен следующий анализ:.
    - Данные были категоризованы путем добавления новых столбцов.
    - По данным оценки пользователей было найдено количество игр по каждой категории. Чаще всего встречается средняя оценка - 4081 раза, если не брать в расчет неизвестные значения.
    - По данным оценки критиков было найдено количество игр по каждой категории. Чаще всего встречается средняя оценка - 5422 раза, если не брать в рвсрасчетчет неизвестные значения.
    - По данным оценки пользователей и критиков было найдено количество игр по каждой категории.
    - Были найдены 7 лучших платформ по количеству игр. Лидером стала платформа: PS2 - 2127 игры, на втором месте платформа: DS - 2120.